<a href="https://colab.research.google.com/github/thisisraj57/Kanishka-Raj_Assignment_Konproz/blob/main/Kanishka_Raj_Assignment_Konproz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VANILLA RAG Q&A BOT

This project is a Retrieval-Augmented Generation (RAG) Q&A bot built with Streamlit. It allows users to ask questions based on documents stored in a FAISS vectorstore, leveraging Google Generative AI for natural language understanding and response generation.

## Features

- **Interactive Q&A Interface**: Users can submit queries and receive answers based on embedded document context.
- **History Tracking**: Keep track of previous queries and responses.
- **Responsive Design**: Built with a user-friendly layout using Streamlit.

## Prerequisites

Before running the application, make sure you have the following:

- Python 3.x
- Google API key with access to Google Generative AI
- Required libraries installed (see Installation section)

## Installation

2. Install the necessary packages using pip:

In [ ]:
#Necessary Liberaries to be installed
!pip install streamlit
!pip install google-generativeai
!pip install langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install -U langchain-community
!pip install faiss-cpu
!pip install dspy

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.1
    Uninstalling pydantic_core-2.14.1:
      Successfully uninstalled pydantic_core-2.14.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.0
    Uninstalling pydantic-2.5.0:
      Successfully uninstalled pydantic-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy-ai 2.4.5 requires pydantic==2.5.0, but you have pydantic 2.9.2 which is incompatible.
  Using cached pydantic-2.5.0-py3-none-any.whl.metadata (

In [ ]:
import getpass
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS  # Import FAISS vectorstore

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here: ")

# Load PDF document
loader = PyPDFLoader("/content/drive/MyDrive/Kanishka Raj_Assignment_Konproz/G S T Smart Guide.pdf")
data = loader.load()

# Split the text into manageable chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
documents = text_splitter.split_documents(data)

# Initialize the embedding
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create FAISS vectorstore from the documents and embeddings
docsearch = FAISS.from_documents(documents, embeddings)

# Save the FAISS index to disk
faiss_index_path = "/content/drive/MyDrive/Kanishka Raj_Assignment_Konproz/faissdb"
docsearch.save_local(faiss_index_path)

print(f"FAISS index saved to: {faiss_index_path}")

Provide your Google API key here: ··········
FAISS index saved to: /content/drive/MyDrive/Kanishka Raj_Assignment_Konproz/faissdb


In [ ]:
%%writefile app.py
import getpass
import os
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here: ")


st.set_page_config(page_title="VANILLA RAG Q&A BOT", page_icon="🧠", layout="centered")
st.markdown(
    '<p style="text-align: center;"><img src="https://cdn.pixabay.com/photo/2024/05/05/16/30/ai-generated-8741448_1280.jpg" alt="Logo" width="200"></p>',
    unsafe_allow_html=True
)
st.markdown("<h1 style='text-align: center; color: black;'>🧠VANILLA RAG Q&A BOT🧠</h1>", unsafe_allow_html=True)

st.sidebar.markdown(
    """
    <div style="background-color: #003366; padding: 9px; border-radius: 9px; text-align: center;">
        <h3 style="color: white;">🌟 "Creativity is intelligence having fun." - Albert Einstein 🌟</h3>
    </div>
    """,
    unsafe_allow_html=True
)
st.sidebar.image(
    "https://e7.pngegg.com/pngimages/799/987/png-clipart-computer-icons-avatar-icon-design-avatar-heroes-computer-wallpaper-thumbnail.png",
    width=100
)
st.sidebar.markdown("### Status")
st.sidebar.markdown("🟢 Online")

st.sidebar.title("About")
st.sidebar.info("This bot uses retrieval-augmented generation (RAG) to answer questions based on documents stored in FAISS.")
st.sidebar.header("👩‍💼 Developer Details")
st.sidebar.write("Developed by: Kanishka Raj")
st.sidebar.write("Contact Details: +91-9612223176")
st.sidebar.markdown("---")

# Initialize session state for query history and responses
if "query_history" not in st.session_state:
    st.session_state.query_history = []
if "response_history" not in st.session_state:
    st.session_state.response_history = []
if "response" not in st.session_state:
    st.session_state.response = None

# Path to the pre-generated FAISS index
faiss_index_path = "/content/drive/MyDrive/Kanishka Raj_Assignment_Konproz/faissdb"

# Initialize the embedding
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load the FAISS vector store
with st.spinner("Loading FAISS index..."):
    docsearch = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    st.success("FAISS vector store loaded successfully!")

# Define the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0, max_tokens=None, timeout=None)

st.subheader("Ask your Query")
with st.form(key="query_form"):
    query = st.text_input("Type your query here...")
    submit_button = st.form_submit_button(label="➡️ Submit")
    delete_button = st.form_submit_button(label="🗑️ Delete")

# System prompt to guide the LLM
system_prompt = ("""
    You are an AI system specifically designed for question-answering tasks. Your primary goal is to provide accurate and relevant answers based on the context provided to you.

    Please adhere to the following instructions:
    - Use only the information from the retrieved context to formulate your answers. Do not rely on external knowledge or assumptions.
    - If a user asks a question that falls outside the context you have, respond with: "I don't have information about this. Please rephrase your query." Ensure this response is clear and polite.
    - Strive to keep your answers clear and to the point. Avoid unnecessary jargon and provide explanations only when relevant to the context.
    - If appropriate, encourage users to ask follow-up questions or provide additional context to facilitate better answers.
    - If you encounter a request that is ambiguous or unclear, ask for clarification to ensure you can provide the best possible response.
    - Maintain a professional and friendly tone in all interactions. Aim to create a positive user experience.
    - Do not store or recall any personal information or past interactions unless explicitly stated in the context.

    "\n\n"
    "{context}"

    """)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Process the query on submit
if submit_button and query:
    with st.spinner("Retrieving relevant information..."):
        question_answer_chain = create_stuff_documents_chain(llm, prompt_template)
        retriever = docsearch.as_retriever()
        relevant_docs = retriever.get_relevant_documents(query)

        top_k = 5
        if len(relevant_docs) > top_k:
            relevant_docs = relevant_docs[:top_k]

        rag_chain = create_retrieval_chain(retriever, question_answer_chain)
        response = rag_chain.invoke({"input": query})

        st.session_state.query_history.append(query)
        st.session_state.response_history.append(response["answer"])
        st.session_state.response = response

        st.markdown("### Answer:")
        st.write(response["answer"])

if delete_button and st.session_state.response_history:
    st.session_state.query_history.pop(-1)
    st.session_state.response_history.pop(-1)
    st.session_state.response = None
    st.success("Previous response deleted!")

if st.session_state.response:
    with st.sidebar.expander("Relevant context retrieved"):
        st.write(st.session_state.response.get("context", "No additional context available."))

with st.sidebar.expander("Query and Answer History"):
    if st.session_state.query_history:
        for i, (q, r) in enumerate(zip(st.session_state.query_history, st.session_state.response_history)):
            st.write(f"**Query {i + 1}:** {q}")
            st.write(f"**Answer {i + 1}:** {r}")
            st.markdown("---")
    else:
        st.write("No history available.")

st.markdown(
    """
    <style>
    .stTextInput { border-radius: 10px; }
    .stButton button { background-color: #4CAF50; color: white; }
    </style>
    """,
    unsafe_allow_html=True
)

Overwriting app.py


In [ ]:
!npm install localtunnel
import sys
import os
import urllib


py_file_location = "/content/app.py"
sys.path.append(os.path.abspath(py_file_location))

print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501


up to date, audited 23 packages in 566ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Password/Enpoint IP for localtunnel is: 34.16.226.129
your url is: https://eighty-moose-brush.loca.lt
^C
